In [1]:
# Importing Marker related modules
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import save_output
from marker.config.parser import ConfigParser

# Importing Chroma related modules
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
## This import gave the following error: Error: DLL load failed while importing cygrpc... . It
## was solved by installing the latest Visual C++ Redistribuable version 
from langchain_chroma import Chroma 

# Importing miscellaneous
import os

# 1. Doelstellingen

Gegeven een vraag willen we de context van de LLM uitbreiden met informatie — met andere woorden, chunks — die inhoudelijk verband houden met de vraag. Daarbij komen twee aspecten kijken:
1. Waar worden de chunks opgeslagen?
2. Hoe wordt het verband tussen de vraag en de relevante chunks gelegd?

Het antwoord op beide vragen ligt bij een essentiële component van de RAG-pijplijn: de datastore. Dit is een vector-databank die tekstuele informatie opslaat in de vorm van embeddings — numerieke vectoren die de betekenis van een tekst representeren. Om deze representaties te verkrijgen, verwertkt de datastore de chunks met een embeddingsmodel, een gespecialiseerd taalmodel dat ontworpen is om semantische informatie te vatten in vectorvorm.

Wanneer een gebruiker een vraag stelt, wordt ook die vraag via hetzelfde model omgezet naar een embedding. Deze vector wordt vervolgens vergeleken met de eerder opgeslagen chunk-embeddings, op zoek naar semantische gelijkenissen. Die gelijkenissen worden bepaald aan de hand van vectorvergelijkingen die de datastore uitvoert.

In dit Notebook verkennen we [Chroma](https://www.trychroma.com/), een vector-databank die ondersteuning biedt voor de hierboven beschreven werkwijze. We willen het volgende nagaan:
- Hoe wordt een Chroma-databank opgezet ? Is het makkelijk ?
- Op welke manier worden de chunks opgeslaan en opgehaald ?

We gebruiken als embeddingsmodel [BAAI/bge-m3](https://huggingface.co/BAAI/bge-m3). BAAI/bge-m3 is een meertalig embeddingsmodel dat ondersteuning biedt voor zowel het Engels als het Nederlands (zie [hier](https://huggingface.co/BAAI/bge-m3/discussions/29)). We willen het volgende nagaan:
- Retourneert Chroma, door gebruik te maken van BAAI/bge-m3, dezelfde chunks ongeacht de taal van de vraag en de taal van de informatiebron

# 2. Methodologie

Het vak Data Science & AI beschikt over een studiewijzer zowel in het Nederlands als in het Engels. Daarom zullen we in dit verder werken met de studiewijzers van dit vak.

Voor elk van deze studiewijzers zullen we vragen stellen, zowel in de taal van de studiewijzer zelf als in de andere taal. Vervolgens onderzoeken we of de teruggegeven chunks dezelfde zijn, ongeacht de gebruikte taal.

# 3. Uittesten

In [2]:
# Defining path to pdfs/markdowns
md_path = './markdowns/'
pdf_path = './pdfs/'

## 3.1. Tekstextractie

Dit gedeelte hoeft enkel uitgevoerd te worden indien de studiewijzers nog niet omgezet werden.

In [3]:
config = { 
    'output_format': 'markdown',
    'disable_image_extraction': True,
}

config_parser = ConfigParser(config)

In [4]:
converter = PdfConverter(
    artifact_dict=create_model_dict(),
    config=config_parser.generate_config_dict(),
)

Loaded layout model s3://layout/2025_02_18 on device cpu with dtype torch.float32
Loaded texify model s3://texify/2025_02_18 on device cpu with dtype torch.float32
Loaded recognition model s3://text_recognition/2025_02_18 on device cpu with dtype torch.float32
Loaded table recognition model s3://table_recognition/2025_02_18 on device cpu with dtype torch.float32
Loaded detection model s3://text_detection/2025_02_28 on device cpu with dtype torch.float32
Loaded detection model s3://inline_math_detection/2025_02_24 on device cpu with dtype torch.float32


In [5]:
for e in os.scandir(pdf_path):
    if e.is_file():
        rendered = converter(e.path)
        save_output(rendered, md_path, e.name[:-4])

Running OCR Error Detection: 100%|████████████████████████████████████████████████████| 12/12 [00:12<00:00,  1.00s/it]
Detecting bboxes: 0it [00:00, ?it/s]
C:\ProgramData\anaconda3\envs\Bachelorproef_onderzoek\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Running OCR Error Detection: 100%|██████████████████████████████████████████████████████| 8/8 [00:08<00:00,  1.04s/it]
Detecting bboxes: 0it [00:00, ?it/s]
C:\ProgramData\anaconda3\envs\Bachelorproef_onderzoek\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Recognizing tables: 100%|█████████████████████████████

## 3.2. Chunking

In [15]:
# Opening Markdown files
with open(os.path.join(md_path, 'Studiewijzer_AI_Data_Science_NL.md'), 'r', encoding='utf-8') as file:
    studiewijzer_nl = file.read()

with open(os.path.join(md_path, 'Studiewijzer_AI_Data_Science_ENG.md'), 'r', encoding='utf-8') as file:
    studiewijzer_eng = file.read()

In [16]:
# Defining splitter
headers_to_split_on = [
    ("#", "h1"),
    ("##", "h2"),
    ("###", "h3"),
    ("####", "h4"),
]

md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on) 

In [17]:
# Creating chunks
studiewijzer_nl_chunks = md_splitter.split_text(studiewijzer_nl) 
studiewijzer_eng_chunks = md_splitter.split_text(studiewijzer_eng) 

## 3.3. Indexing

In [18]:
# Creating embedding model
embedding_model = 'BAAI/bge-m3'
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

In [19]:
# Creating datastores
eng_store = Chroma(
    collection_name='eng_collection',
    embedding_function=embeddings,
)

nl_store = Chroma(
    collection_name='nl_collection',
    embedding_function=embeddings,
)

In [20]:
# Inserting chunks in their respective stores
for i, doc in enumerate(studiewijzer_nl_chunks):
    doc.id = str(i + 1)

for i, doc in enumerate(studiewijzer_eng_chunks):
    doc.id = str(i + 1)
    
nl_store.add_documents(studiewijzer_nl_chunks)
eng_store.add_documents(studiewijzer_eng_chunks)

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80']

## 3.4. Querying

In [30]:
# Helpfunction that will print the individual chunks together with 
# their similarity score
def get_chunks_info(chunks):
    for i, t in enumerate(chunks):
        chunk, score = t
        
        print(f'--------- CHUNK {i + 1} ---------\n')
        print(chunk.page_content)
        print(f'\n*** SCORE: {score} ***\n')


In [46]:
# Defining question pairs
questions = [
    ('Welke programma\'s zijn nodig voor dit vak ?', 'Which softwares are needed for this course ?'),
    ('Worden online examens georganiseerd voor dit vak ?', 'Are online exams organized for this course?'),
    ('Wie zijn de lectoren van dit vak ?', 'Who are the teachers of this course ?'),
    ('In welk week wordt er gestart met statistische toetsen ?', 'In which week do statistical tests begin ?'),
    ('Op welk boek is dit cursus gebaseerd ?', 'On which book is this course based ?')
]

### 3.4.1. NL. Store

In [52]:
# Printing the questions, together with te retrieved chunks
for question_pair in questions:
    nl_question, eng_question = question_pair
    nl_chunks = nl_store.similarity_search_with_score(nl_question, k = 2)
    eng_chunks = nl_store.similarity_search_with_score(eng_question, k = 2)

    print(f'********* Vraag: {nl_question} *********')
    get_chunks_info(nl_chunks)

    print(f'********* Question: {eng_question} *********')
    get_chunks_info(eng_chunks)

********* Vraag: Welke programma's zijn nodig voor dit vak ? *********
--------- CHUNK 1 ---------

Als je het voor jezelf nuttig vindt om op je eigen laptop te werken, dan kan dat ook (maar het is niet vereist om deze cursus met succes af te werken). In dat geval heb je volgende software nodig:  
- Python
- Visual Studio Code, met extensies:
- Python, Pylance (Microsoft)
- Jupyter, Jupyter Keymap, Jupyter Notebook Rendering (Microsoft)
- Optioneel: GitLens (GitKraken), Markdown All in One (Yu Zhang)
- Git & een Github-account (we veronderstellen dat je hier al over beschikt)  
Installeer de software op Windows bv. met winget (in een CMD of PowerShell terminal met Administrator-rechten):  
```
> winget install Python.Python.3.12
> winget install Microsoft.VisualStudioCode
```
Gebruikers van Linux en macOS kunnen de respectievelijke package managers van hun OS (apt, dnf, HomeBrew, ...) gebruiken voor de installatie. Op Linux is Python ongetwijfeld reeds geïnstalleerd en hoef je dus enke

### 3.4.2. ENG. Store

In [53]:
# Printing the questions, together with te retrieved chunks
for question_pair in questions:
    nl_question, eng_question = question_pair
    eng_chunks = eng_store.similarity_search_with_score(eng_question, k = 2)
    nl_chunks = eng_store.similarity_search_with_score(nl_question, k = 2)
    
    print(f'********* Question: {eng_question} *********')
    get_chunks_info(eng_chunks)
    
    print(f'********* Vraag: {nl_question} *********')
    get_chunks_info(nl_chunks)

********* Question: Which softwares are needed for this course ? *********
--------- CHUNK 1 ---------

If you find it useful to work on your own laptop, you may do so (but it is not required for the successful completion of this course). In this case, you will need the following software:  
- Python
- Visual Studio Code, with extensions:
- Python, Pylance (Microsoft)
- Jupyter, Jupyter Keymap, Jupyter Notebook Rendering (Microsoft)
- Optionally: GitLens (GitKraken), Markdown All in One (Yu Zhang)
- Git & a Github-account  
We have prepared a [Github repository \(https://github.com/HoGentTIN/dsai-en-labs\)](https://github.com/HoGentTIN/dsai-en-labs) for the Python lab commands that contains the assignments and associated datasets, as well as sample code. Please make a local clone of this repository (or download as ZIP) and open the directory in Visual Studio Code. By the way, VS Code itself will offer to install the most important extensions as soon as you open a Python script or Jupyt

# 4. Besluit

Chroma is een zeer gebruiksvriendelijke vector-databank. Het opzetten ervan verloopt in slechts enkele regels code, en ook het opslaan en ophalen van chunks is bijzonder eenvoudig.

Door gebruik te maken van het embeddingsmodel BAAI/bge-m3, retourneert Chroma consistente chunks, ongeacht de taal van de vraag; beide talen hebben steeds minstens één chunk gemeenschappelijk. Wel viel op dat sommige chunks een deel van het antwoord misten. Dit is te wijten aan de omzetting van PDF naar Markdown, wat de structuur van de tekst beïnvloedt en daardoor ook de opbouw van de gegenereerde chunks.